## Load & Chunk

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
    ),
)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = loader.load()
chunks = text_splitter.split_documents(documents)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
print(f"Total characters: {len(documents[0].page_content)}")
print(f"Total chunks: {len(chunks)}")
print(chunks[0].page_content)

Total characters: 43131
Total chunks: 66
LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results

## Embed

In [4]:
from sentence_transformers import SentenceTransformer
from typing import List


class SentenceTransformerWrapper:
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t).tolist() for t in texts]

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode(query).tolist()


embedder = SentenceTransformerWrapper("all-MiniLM-L6-v2")

/Users/mikolajpiorczynski/miniconda3/envs/mini-rag-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings = embedder.embed_documents([c.page_content for c in chunks[:5]])
len(embeddings), len(embeddings[0])

(5, 384)

In [6]:
embedding = embedder.embed_query(chunks[0].page_content)
len(embedding)

384

## Store

In [7]:
from langchain_chroma import Chroma

db_directory = "data/db"
vector_store = Chroma(
    collection_name="rag-in-5-minutes",
    embedding_function=embedder,
    persist_directory=db_directory,
)

In [7]:
uuids = vector_store.add_documents(documents=chunks, embeddings=embeddings)

In [8]:
# from uuid import uuid4
# from langchain_core.documents import Document

# for i, chunk in enumerate(chunks):
#     doc = Document(
#         id=i,
#         page_content=chunk.page_content,
#         metadata={"source": chunk.metadata["source"], "chunk_index": i},
#     )
#     vector_store.add_documents(documents=[doc], ids=[str(uuid4())])

In [10]:
# id = uuids[0]
id = "a4b8f413-e8d4-47cd-b9a4-63ce18808a46"
vector_store.get(id)

{'ids': ['a4b8f413-e8d4-47cd-b9a4-63ce18808a46'],
 'embeddings': None,
 'documents': ['LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for fu

In [11]:
vector_store.delete(id)

In [12]:
vector_store.get(id)

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### Retrieve

In [11]:
query = "What is Task Decomposition?"

In [12]:
results = vector_store.similarity_search_with_score(
    query,
    k=1,
    filter={"source": "https://lilianweng.github.io/posts/2023-06-23-agent/"},
)
for res, score in results:
    print(f"(score: {score}) {res.page_content}")

(score: 0.9169834852218628) Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.


In [13]:
results = vector_store.similarity_search_by_vector(
    embedding=embedder.embed_query(query),
    k=1,
    filter={"source": "https://lilianweng.github.io/posts/2023-06-23-agent/"},
)
for res in results:
    print(f"{res.page_content}")

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.


## Retrieve and Generate

In [ ]:
# todo